In [ ]:
# !pip install sqlalchemy psycopg2-binary geoalchemy2

In [12]:
import psycopg2

# 1) Connect and set autocommit so DDL runs immediately
conn = psycopg2.connect(
    dbname='indego',
    user='postgres',
    password='cfcf133201',
    host='localhost',
    port='5432'
)
conn.autocommit = True

with conn.cursor() as cur:
    # 2) Ensure the schema exists
    cur.execute("CREATE SCHEMA IF NOT EXISTS indego;")
    
    # 3) Define table DDL (same for both quarters)
    table_ddl = """
        trip_id TEXT,
        duration INTEGER,
        start_time TIMESTAMP,
        end_time TIMESTAMP,
        start_station TEXT,
        start_lat FLOAT,
        start_lon FLOAT,
        end_station TEXT,
        end_lat FLOAT,
        end_lon FLOAT,
        bike_id TEXT,
        plan_duration INTEGER,
        trip_route_category TEXT,
        passholder_type TEXT,
        bike_type TEXT
    """

    # 4) Loop over Q3 2021 and Q3 2022
    datasets = [
        ('trips_2021_q3', '../data/indego-trips-2021-q3.csv'),
        ('trips_2022_q3', '../data/indego-trips-2022-q3.csv'),
    ]

    for table_name, csv_path in datasets:
        # a) Create table if it doesn't exist
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS indego.{table_name} (
                {table_ddl}
            );
        """)
        
        # b) Load CSV via COPY
        with open(csv_path, 'r') as f:
            cur.copy_expert(
                f"COPY indego.{table_name} FROM STDIN WITH CSV HEADER",
                f
            )
        print(f"Loaded {csv_path} into indego.{table_name}")

conn.close()
print("All done.")


Loaded ../data/indego-trips-2021-q3.csv into indego.trips_2021_q3
Loaded ../data/indego-trips-2022-q3.csv into indego.trips_2022_q3
All done.


In [9]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geography

# — read GeoJSON via Fiona instead of Pyogrio —
gdf = gpd.read_file(
    "../data/station_status.geojson",
    engine="fiona"
)

# rename geometry column to "geog"
gdf = gdf.set_geometry("geometry").rename_geometry("geog")

# build your connection string dynamically (from env‐vars or dict)
import os
DB = {
    "user":     os.getenv("DB_USER",     "postgres"),
    "password": os.getenv("DB_PASSWORD", "cfcf133201"),
    "host":     os.getenv("DB_HOST",     "localhost"),
    "port":     os.getenv("DB_PORT",     "5432"),
    "database": os.getenv("DB_NAME",     "indego"),
}
conn_str = (
    f"postgresql+psycopg2://"
    f"{DB['user']}:{DB['password']}@"
    f"{DB['host']}:{DB['port']}/{DB['database']}"
)
engine = create_engine(conn_str)

# ensure PostGIS + schema
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS indego;"))

# push to PostGIS
gdf.to_postgis(
    name="station_statuses",
    con=engine,
    schema="indego",
    if_exists="replace",
    index=False,
    dtype={"geog": Geography("POINT", srid=4326)}
)

print("station_statuses loaded into indego!")


station_statuses loaded into indego!
